In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


In [2]:
with open("data/x_train.txt") as f:
    lines = [x[:-1] for x in f.readlines()]
f.close()


In [3]:
with open("data/y_train.txt") as f:
    labels = [x[:-1] for x in f.readlines()]
f.close()


In [4]:
lines[0]

'The Russon mill, a water mill is located in the street of de la Brosse. The former owner has restored it. The mill was a component of the outbuildings of the Farm of Roquemont. It was used as a mill only for a few months and was otherwise used as accommodation for the farm hands.'

In [5]:
[x + y for x, y in zip(lines[0].split()[:-1], lines[0].split()[1:])]


['TheRusson',
 'Russonmill,',
 'mill,a',
 'awater',
 'watermill',
 'millis',
 'islocated',
 'locatedin',
 'inthe',
 'thestreet',
 'streetof',
 'ofde',
 'dela',
 'laBrosse.',
 'Brosse.The',
 'Theformer',
 'formerowner',
 'ownerhas',
 'hasrestored',
 'restoredit.',
 'it.The',
 'Themill',
 'millwas',
 'wasa',
 'acomponent',
 'componentof',
 'ofthe',
 'theoutbuildings',
 'outbuildingsof',
 'ofthe',
 'theFarm',
 'Farmof',
 'ofRoquemont.',
 'Roquemont.It',
 'Itwas',
 'wasused',
 'usedas',
 'asa',
 'amill',
 'millonly',
 'onlyfor',
 'fora',
 'afew',
 'fewmonths',
 'monthsand',
 'andwas',
 'wasotherwise',
 'otherwiseused',
 'usedas',
 'asaccommodation',
 'accommodationfor',
 'forthe',
 'thefarm',
 'farmhands.']

In [6]:
CONTEXT_SIZE = 2
EMBEDDING_DIM = 10
# We will use Shakespeare Sonnet 2
test_sentence = """When forty winters shall besiege thy brow,
And dig deep trenches in thy beauty's field,
Thy youth's proud livery so gazed on now,
Will be a totter'd weed of small worth held:
Then being asked, where all thy beauty lies,
Where all the treasure of thy lusty days;
To say, within thine own deep sunken eyes,
Were an all-eating shame, and thriftless praise.
How much more praise deserv'd thy beauty's use,
If thou couldst answer 'This fair child of mine
Shall sum my count, and make my old excuse,'
Proving his beauty by succession thine!
This were to be new made when thou art old,
And see thy blood warm when thou feel'st it cold.""".split()
# we should tokenize the input, but we will ignore that for now
# build a list of tuples.
# Each tuple is ([ word_i-CONTEXT_SIZE, ..., word_i-1 ], target word)
ngrams = [
    ([test_sentence[i - j - 1] for j in range(CONTEXT_SIZE)], test_sentence[i])
    for i in range(CONTEXT_SIZE, len(test_sentence))
]
# Print the first 3, just so you can see what they look like.
print(ngrams[:3])


[(['forty', 'When'], 'winters'), (['winters', 'forty'], 'shall'), (['shall', 'winters'], 'besiege')]


In [8]:
vocab = set(test_sentence)
word_to_ix = {word: i for i, word in enumerate(vocab)}


class NGramLanguageModeler(nn.Module):
    def __init__(self, vocab_size, embedding_dim, context_size):
        super(NGramLanguageModeler, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(context_size * embedding_dim, 128)
        self.linear2 = nn.Linear(128, vocab_size)

    def forward(self, inputs):
        embeds = self.embeddings(inputs).view((1, -1))
        out = F.relu(self.linear1(embeds))
        out = self.linear2(out)
        log_probs = F.log_softmax(out, dim=1)
        return log_probs


In [10]:
losses = []
loss_function = nn.NLLLoss()
model = NGramLanguageModeler(len(vocab), EMBEDDING_DIM, CONTEXT_SIZE)
optimizer = optim.SGD(model.parameters(), lr=0.001)


In [11]:
for epoch in range(10):
    total_loss = 0
    for context, target in ngrams:

        # Step 1. Prepare the inputs to be passed to the model (i.e, turn the words
        # into integer indices and wrap them in tensors)
        context_idxs = torch.tensor([word_to_ix[w] for w in context], dtype=torch.long)

        # Step 2. Recall that torch *accumulates* gradients. Before passing in a
        # new instance, you need to zero out the gradients from the old
        # instance
        model.zero_grad()

        # Step 3. Run the forward pass, getting log probabilities over next
        # words
        log_probs = model(context_idxs)

        # Step 4. Compute your loss function. (Again, Torch wants the target
        # word wrapped in a tensor)
        loss = loss_function(
            log_probs, torch.tensor([word_to_ix[target]], dtype=torch.long)
        )

        # Step 5. Do the backward pass and update the gradient
        loss.backward()
        optimizer.step()

        # Get the Python number from a 1-element Tensor by calling tensor.item()
        total_loss += loss.item()
    losses.append(total_loss)
print(losses)  # The loss decreased every iteration over the training data!


[520.9734246730804, 518.6919302940369, 516.4234900474548, 514.1668071746826, 511.9210991859436, 509.68646216392517, 507.4624557495117, 505.24717807769775, 503.0390512943268, 500.83824825286865]
